# Goal: Investigate birth and death among closed classes of words
Load and Pre-Process   
1. Load all the gzipped ngrams data into the notebook  
2. Pre-process Google Ngrams database so that there is only the following [Google Tags](https://books.google.com/ngrams/info)
    - _PRON_	pronoun
    - _DET_	determiner or article
    - _ADP_	an adposition: either a preposition or a postposition
    - _CONJ_	conjunction
    - _PRT_	particle  
    (These tags can only be appended to a word (she_PRON), no stand alones (\_PRON\_))
3. Save as JSON file  

In [1]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

## Load and pre-process 

In [2]:
import os
import gzip
import re
from unidecode import unidecode
#For the Google POS tagging
underscore = re.compile('_{1}')

In [3]:
import string
PUNCTUATION = set(char for char in string.punctuation).union({'“','”'})
DIGITS = set(string.digits)
VOWELS = set("aeiouyAEIOUY")
#Excluding '_' (underscore) from DASHES precludes the tagged 1grams "_NOUN", add it to also include the tagged 1grams
DASHES = {'—','–','—','―','‒','-','_'}
PUNCTUATION.difference_update(DASHES)
STOPS = PUNCTUATION.union(DIGITS)
GOOGLE_TAGS = {'PRON','DET','ADP','CONJ','PRT'}

In [4]:
def open_gzip(directory,file_path):
    with gzip.open(directory+file_path,'r') as f_in:
        rows = [x.decode('utf8').strip() for x in f_in.readlines()]
    return rows

In [5]:
def csv2tuple(string):
    year,match_count,volume_count = tuple(string.split(','))
    return int(year),int(match_count),int(volume_count)

In [6]:
def save_json(dictionary,directory,file_path):
    output = file_path+'.json'
    if len(dictionary)>0:
        with open(directory+output, 'w') as f_out:
            json.dump(dictionary, f_out)
        print('SAVED: ',output,len(dictionary))
    else:
        print('unigram dict empty',output)

In [7]:
def unigram_tests(unigram):
    #Checks each character in the unigram against the characters in the STOP set. (character level filtering) - no punctuation or digits allowed
    if set(unigram).intersection(STOPS):
        return False
    
    #must have a vowel (presupposes that it must also have a letter of the alphabet inside)
    if not set(unigram).intersection(VOWELS):
        return False #Rewrite the alphabet one, i think this is better
    
    #Words cannot start or end with dashes
    if unigram[0] in DASHES or unigram[-1] in DASHES:
        return False
    
    #Exclude words with more than one underscore, can make this != to only include tagged words
    if len(underscore.findall(unigram))>1:
        return False
    
    #must have 0 non-english letters
    test = unidecode(unigram, errors='replace')
    if test != unigram:
        return False
    
    #Can implement more tests here if you need to do more filtering
    
    else:
        return True

In [8]:
def preprocess_ngrams(directory,file_path):
    
    rows = open_gzip(directory,file_path)
    ngram_dict = dict()

    #This implementation uses {1gram:{year:match_count ...} ...}
    for row in tqdm(rows):
        columns = row.split('\t')
        #unigram is the first entry, the rest of the entries are of the form year,match_count,volume_count\t n times, where n is variable each line
        
        unigram = columns[0]
        if len(underscore.findall(unigram))==1: #One and only one underscore allowed
            word_tag = underscore.split(unigram) # list of [word,tag]
            #checks if tag is Google tag
            if word_tag[1] in GOOGLE_TAGS:
                #Removes the tag before processing unigram string
                unigram = word_tag[0].lower().strip()+'_'+word_tag[1]
                if unigram_tests(unigram):
                    #Parse the new entry and create a list of records in form [...[year, match_count]...]
                    records = dict()
                    #the first entry in columns is word so that is exluded
                    for entry in columns[1:]:
                        year,match_count,volume_count = csv2tuple(str(entry))
                        if year>1800 and volume_count>1:
                            records[year] = match_count

                    #Modify the dictionary if new entry is already there, else just add it as a new unigram:records to the dict
                    if unigram in ngram_dict.keys():
                        #accessing the ngram dictionary and seeing if each year is present, if so add match count, else add a new record entry to the dictionary.
                        for yr, match_ct in records.items(): #each record should be of the form {year, match_count}
                            #If the year in the new record is in the dict for this 1gram, then find where it is.
                            if yr in ngram_dict[unigram].keys():
                                ngram_dict[unigram][yr] += match_ct
                            else:
                                #This just adds the record to the end, will need to sort later
                                ngram_dict[unigram][yr] = match_ct
                    else:
                        ngram_dict[unigram] = records
    
    #Save as JSON
    save_json(ngram_dict,directory,file_path[:-3]+'_CLOSED_CLASSES')

## Run Everything

In [9]:
%%time
directory = './Ngrams/'
files = os.listdir(directory)
for file_path in files:
    if '.gz' in file_path:
        preprocess_ngrams(directory,file_path)

100%|██████████| 2396510/2396510 [00:08<00:00, 273583.59it/s]


unigram dict empty 1-00000-of-00024_CLOSED_CLASSES.json


100%|██████████| 3100658/3100658 [00:11<00:00, 272696.32it/s]


SAVED:  1-00015-of-00024_CLOSED_CLASSES.json 13055


100%|██████████| 3109631/3109631 [00:14<00:00, 210005.80it/s]


...

SAVED:  1-00012-of-00024_CLOSED_CLASSES.json 12206


100%|██████████| 3427775/3427775 [00:18<00:00, 186505.40it/s]


SAVED:  1-00020-of-00024_CLOSED_CLASSES.json 77955


100%|██████████| 3384843/3384843 [00:13<00:00, 250183.21it/s]


SAVED:  1-00007-of-00024_CLOSED_CLASSES.json 9092
CPU times: user 11min 17s, sys: 1min 27s, total: 12min 44s
Wall time: 14min 27s
